In [2]:
## Feed Forward Neural Network
import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer


from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
from torch.utils.data import TensorDataset, DataLoader

# Load data
data = load_breast_cancer()
X, y = data.data, data.target

# Standardize
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train/Test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Convert to tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

# Dataloaders
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)


In [3]:
import torch.nn as nn

class FeedforwardNN(nn.Module):
    def __init__(self, input_dim):
        super(FeedforwardNN, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.network(x)

In [4]:
model = FeedforwardNN(X.shape[1])
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(100):
    for xb, yb in train_loader:
        pred = model(xb)
        loss = criterion(pred, yb)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [5]:
with torch.no_grad():
    preds = model(X_test)
    predicted = (preds > 0.5).float()
    accuracy = (predicted == y_test).float().mean()
    print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.9825


In [6]:
model_size = sum(p.numel() for p in model.parameters())
print(f"Model size (total parameters): {model_size}")

Model size (total parameters): 641


In [7]:
#Diabetes

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error

# Load and prepare data
data = load_diabetes()
X, y = data.data, data.target

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

train_ds = TensorDataset(X_train, y_train)
train_dl = DataLoader(train_ds, batch_size=32, shuffle=True)

# Define feedforward network
class Net(nn.Module):
    def __init__(self, input_dim):
        super(Net, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )

    def forward(self, x):
        return self.fc(x)

model = Net(X.shape[1])
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training
for epoch in range(100):
    for xb, yb in train_dl:
        pred = model(xb)
        loss = loss_fn(pred, yb)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

# Evaluation
with torch.no_grad():
    preds = model(X_test)
    mse = mean_squared_error(y_test, preds)
    r2 = r2_score(y_test, preds)

print(f"Mean Squared Error: {mse:.2f}")
print(f"R² Score: {r2:.2f}")

# Model size
model_size = sum(p.numel() for p in model.parameters())
print(f"Model size (total parameters): {model_size}")

Mean Squared Error: 3151.94
R² Score: 0.41
Model size (total parameters): 2817


In [8]:
from torchviz import make_dot
from torch.utils.tensorboard import SummaryWriter

# Create a dummy input matching your feature size:
dummy_input = torch.randn(1, X_train.shape[1])

# --- A) TorchViz / Graphviz ---
# This will produce "diabetes_model_architecture.png" in your working directory.
y = model(dummy_input)
dot = make_dot(y, params=dict(model.named_parameters()))
dot.format = 'png'
dot.render('diabetes_model_architecture')
print("→ Graphviz visualization saved as diabetes_model_architecture.png")

# --- B) TensorBoard ---
# Logs to "./runs/diabetes_experiment" — run `tensorboard --logdir=runs` to view.
writer = SummaryWriter('runs/diabetes_experiment')
writer.add_graph(model, dummy_input)
writer.close()
print("→ TensorBoard log written to runs/diabetes_experiment")

ExecutableNotFound: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH